In [1]:
import pandas as pd
import numpy as np
import requests
import os

from collections import Counter

# Train

In [2]:
maindir = os.getcwd()
os.chdir("/../../datasets/aclImdb/train")
os.listdir()

['pos',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt',
 'neg',
 'labeledBow.feat',
 'unsupBow.feat',
 'unsup']

In [3]:
# f_neg_urls = open("urls_neg.txt", "r")
# neg_urls = []
# for i in f_neg_urls:
#     neg_urls.append(i.strip("\n"))
#     f_neg_urls = open("urls_neg.txt", "r")
# (f_neg_urls).close()

# f_pos_urls = open("urls_pos.txt", "r")
# pos_urls = []
# for i in f_pos_urls:
#     pos_urls.append(i.strip("\n"))
# (f_pos_urls).close()  

# f_labels = open("labeledBow.feat",'r')
# labels_dirty = f_labels.read().split(" ")
# labels = []
# for i in labels_dirty:
#     labels.append(i.replace("\n",''))
# (f_labels).close()

In [4]:
os.chdir("pos")

In [5]:
pos_files = os.listdir()
pos_reviews = []
for i in pos_files:
    f = open(i,'r')
    rev = f.read()
    pos_reviews.append(rev)
    f.close()

In [6]:
os.chdir("../neg")

In [7]:
neg_files = os.listdir()
neg_reviews = []
for i in neg_files:
    f = open(i,'r')
    rev = f.read()
    neg_reviews.append(rev)
    f.close()

In [8]:
df_pos = pd.DataFrame({"File":pos_files,"Review":pos_reviews,"Label":[1]*len(pos_files)})
df_neg = pd.DataFrame({"File":neg_files,"Review":neg_reviews,"Label":[-1]*len(neg_files)})

In [9]:
df_train = pd.concat([df_pos, df_neg])

In [10]:
df_train.head()

,File,Review,Label
0,2893_10.txt,Walt Disney's CINDERELLA takes a story everybo...,1
1,7944_9.txt,"Have you ever, or do you have, a pet who's bee...",1
2,11725_10.txt,"I suck at gratuitous Boob references, so i'm j...",1
3,1587_10.txt,"Does anyone know, where I can see or download ...",1
4,10297_8.txt,Well not actually. This movie is very entertai...,1


# Test

In [11]:
os.chdir("../../test")

In [12]:
# f_neg_urls = open("urls_neg.txt", "r")
# neg_urls = []
# for i in f_neg_urls:
#     neg_urls.append(i.strip("\n"))
#     f_neg_urls = open("urls_neg.txt", "r")
# (f_neg_urls).close()

# f_pos_urls = open("urls_pos.txt", "r")
# pos_urls = []
# for i in f_pos_urls:
#     pos_urls.append(i.strip("\n"))
# (f_pos_urls).close()  

f_labels = open("labeledBow.feat",'r')
labels_dirty = f_labels.read().split(" ")
labels = []
for i in labels_dirty:
    labels.append(i.replace("\n",''))
(f_labels).close()

In [13]:
os.chdir("pos")

In [14]:
pos_files = os.listdir()
pos_reviews = []
for i in pos_files:
    f = open(i,'r')
    rev = f.read()
    pos_reviews.append(rev)
    f.close()

In [15]:
os.chdir("../neg")

In [16]:
neg_files = os.listdir()
neg_reviews = []
for i in neg_files:
    f = open(i,'r')
    rev = f.read()
    neg_reviews.append(rev)
    f.close()

In [17]:
df_pos = pd.DataFrame({"File":pos_files,"Review":pos_reviews,"Label":[1]*len(pos_files)})
df_neg = pd.DataFrame({"File":neg_files,"Review":neg_reviews,"Label":[-1]*len(neg_files)})

In [18]:
df_test = pd.concat([df_pos, df_neg])

In [19]:
df_test.head()

,File,Review,Label
0,2893_10.txt,"""Rush in Rio"" is, no doubt, one of the most ex...",1
1,8705_10.txt,I have seen a number of horror movies to know ...,1
2,11725_10.txt,I'm a fan of B grade 80s films in which the he...,1
3,9859_8.txt,"I think that Pierre Léaud, or his character, t...",1
4,12409_10.txt,This picture doesn't have any big explosions o...,1


# Feature Set for Train and Test

In [20]:
# Stop words
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

In [21]:
# Positive semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Positive\n;\n; This file contains a list of POSITIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'
pos_words = r.text[len(s)+2:]
pos_words = pos_words.split("\n")

In [22]:
# Negative semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Negative\n;\n; This file contains a list of NEGATIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n\n'
neg_words = r.text[len(s):]
neg_words = neg_words.split("\n")

## Engineering the raw data

In [23]:
# Vectorizing reviews in training set
words_train = (
    df_train.Review.
    str.lower().
    str.replace("[^\w\s]","").
    str.split()
)
bag_of_words_train = words_train.apply(Counter)

In [24]:
# Vectorizing reviews in test set
words_test = (
    df_test.Review.
    str.lower().
    str.replace("[^\w\s]","").
    str.split()
)
bag_of_words_test = words_test.apply(Counter)

In [25]:
# Removing stop words in train
reviews_train = []
for r in words_train:
    good = []
    for w in r:
        if w not in en_stops:
            good.append(w)
    reviews_train.append(good)
reviews_train = pd.Series(reviews_train)

In [26]:
# Removing stop words in test
reviews_test = []
for r in words_test:
    good = []
    for w in r:
        if w not in en_stops:
            good.append(w)
    reviews_test.append(good)
reviews_test = pd.Series(reviews_test)

In [27]:
# Counting positive and negative words in training set
pos_set = set(pos_words)
neg_set = set(neg_words)
posc_train = []
negc_train = []
for r in reviews_train:
    count_pos = len(pos_set.intersection(set(r)))
    count_neg = len(neg_set.intersection(set(r)))
    posc_train.append(count_pos)
    negc_train.append(count_neg)
    

In [28]:
# Counting positive and negative words in test set
pos_set = set(pos_words)
neg_set = set(neg_words)
posc_test = []
negc_test = []
for r in reviews_test:
    count_pos = len(pos_set.intersection(set(r)))
    count_neg = len(neg_set.intersection(set(r)))
    posc_test.append(count_pos)
    negc_test.append(count_neg)

In [29]:
df_train = pd.DataFrame({"Positive_counts":posc_train, 
                         "Negative_counts":negc_train, 
                         "Label": df_train.Label})

In [30]:
df_test = pd.DataFrame({"Positive_counts":posc_test, 
                        "Negative_counts":negc_test,
                        "Label": df_test.Label})

## Saving Data

In [31]:
os.chdir(maindir)
df_train.to_csv("train1.csv")
df_test.to_csv("test1.csv")